Our team attempted to replicate the results that we have discussed in our final report on a real IBM quantum device. However, due to computational constraints, processing one 2x2 patch of an image took approximately one minute, and even after one hour, we were unable to preprocess a single image fully. Additionally, IBM's quantum computer usage is limited to 10 minutes per month, posing significant limitations (because of this we got an error which can be seen in the last cell). Despite these challenges, we successfully wrote the entire code intended for the IBM server, which can be utilized once faster quantum devices become available.

# following is the broad skeleton that we have used in the code written below

# (1) importing modules

In [1]:
#!pip install qiskit_ibm_provider
#!pip install pennylane-qiskit


# (2) initialisation of parameters

# (3) dataset loading

# (4) Neural Network with quantum convolution on real quantum device

# (5) Loading my IBM Quantum account

# (6) Define Quantum Circuit with Qiskit (right now only for 2*2 kernel size)

# (7) Quantum Convolution

# (8) Pre-process Data Using Quantum Convolution

# (9) Train and Evaluate Neural Network

# (10) Plots of the Results from the real IBM device

In [6]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_provider import IBMProvider
from qiskit.tools.monitor import job_monitor

# Set up parameters
n_epochs = 30   # Number of optimization epochs
n_layers = 1    # Number of random layers
n_train = 2    # Size of the train dataset
n_test = 1     # Size of the test dataset

SAVE_PATH = "./"  # Data saving folder
PREPROCESS = True           # If False, skip quantum processing and load data from SAVE_PATH
np.random.seed(0)           # Seed for NumPy random number generator
tf.random.set_seed(0)       # Seed for TensorFlow random number generator

# Load dataset
mnist_dataset = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

# Reduce dataset size
train_images = train_images[:n_train]
train_labels = train_labels[:n_train]
test_images = test_images[:n_test]
test_labels = test_labels[:n_test]

# Normalize pixel values within 0 and 1
train_images = train_images / 255
test_images = test_images / 255

# Add extra dimension for convolution channels
train_images = np.array(train_images[..., tf.newaxis], requires_grad=False)
test_images = np.array(test_images[..., tf.newaxis], requires_grad=False)

# Authenticate IBM Quantum account
IBMQ.save_account('YOUR_API_TOKEN', overwrite=True)
provider = IBMProvider()

# List all available backends
backends = provider.backends()
for backend in backends:
    print(backend)

# Choose an available backend
backend = provider.get_backend('ibm_kyoto')  # Replace with an available backend

# Print selected backend
print("Selected backend:", backend)

# Define quantum circuit
kernel_size = 4  # 2x2 kernel has 4 elements
rand_params2 = np.random.uniform(high=2 * np.pi, size=(n_layers, kernel_size))

def qiskit_circuit(phi):
    """Create a Qiskit quantum circuit for a 2x2 kernel size"""
    qc = QuantumCircuit(kernel_size)
    phi = np.array(phi)  # Ensure phi is a standard NumPy array

    # Encoding classical values into quantum states using Ry rotations
    for i in range(kernel_size):
        qc.ry(np.pi * float(phi[i]), i)  # Convert phi[i] to a float

    # Random quantum circuit (using the rand_params2)
    for layer in range(n_layers):
        for i in range(kernel_size):
            qc.ry(float(rand_params2[layer][i]), i)  # Convert params to float
        for i in range(0, kernel_size, 2):
            qc.cx(i, (i+1) % kernel_size)
    
    qc.measure_all()
    return qc

def execute_circuit(circuit, backend, shots=1024):
    """Execute the quantum circuit on the chosen backend"""
    transpiled_circuit = transpile(circuit, backend)
    job = backend.run(transpiled_circuit, shots=shots)
    job_monitor(job)
    result = job.result()
    counts = result.get_counts()
    return counts

def qiskit_expectation(counts):
    """Calculate expectation values from the counts"""
    shots = sum(counts.values())
    expectation = []
    for i in range(kernel_size):
        exp = 0
        for key, count in counts.items():
            padded_key = key.zfill(kernel_size)  # Pad key to match kernel_size
            if padded_key[i] == '0':
                exp += count
            else:
                exp -= count
        expectation.append(exp / shots)
    return expectation

# Update to use the default.qubit device for Pennylane compatibility
dev = qml.device("default.qubit", wires=kernel_size)

@qml.qnode(dev)
def circuit(phi):
    qc = qiskit_circuit(phi)
    counts = execute_circuit(qc, backend)
    expectations = qiskit_expectation(counts)
    for i in range(len(expectations)):
        qml.RY(expectations[i], wires=i)
    return [qml.expval(qml.PauliZ(i)) for i in range(kernel_size)]

# Quantum convolution function
def quanv(image):
    """Convolves the input image with many applications of the same quantum circuit."""
    out = np.zeros((14, 14, 4))

    # Loop over the coordinates of the top-left pixel of 2x2 squares
    for j in range(0, 28, 2):
        for k in range(0, 28, 2):
            q_results = circuit([
                float(image[j, k, 0]),
                float(image[j, k + 1, 0]),
                float(image[j + 1, k, 0]),
                float(image[j + 1, k + 1, 0])
            ])
            for c in range(4):
                out[j // 2, k // 2, c] = q_results[c]
    return out

# Pre-process data using quantum convolution
if PREPROCESS:
    q_train_images = []
    print("Quantum pre-processing of train images:")
    for idx, img in enumerate(train_images):
        print(f"{idx + 1}/{n_train}        ", end="\r")
        q_train_images.append(quanv(img))
    q_train_images = np.asarray(q_train_images)

    q_test_images = []
    print("\nQuantum pre-processing of test images:")
    for idx, img in enumerate(test_images):
        print(f"{idx + 1}/{n_test}        ", end="\r")
        q_test_images.append(quanv(img))
    q_test_images = np.asarray(q_test_images)

    np.save(SAVE_PATH + "q_train_images.npy", q_train_images)
    np.save(SAVE_PATH + "q_test_images.npy", q_test_images)
else:
    q_train_images = np.load(SAVE_PATH + "q_train_images.npy")
    q_test_images = np.load(SAVE_PATH + "q_test_images.npy")

# Train and evaluate neural network
def MyModel():
    model = keras.models.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

def train_and_evaluate():
    np.random.seed(0)
    tf.random.set_seed(0)
    q_model = MyModel()
    q_history = q_model.fit(
        q_train_images,
        train_labels,
        validation_data=(q_test_images, test_labels),
        batch_size=4,
        epochs=n_epochs,
        verbose=2,
        shuffle=False
    )
    return q_history.history['val_accuracy'], q_history.history['val_loss']

num_runs = 20
val_accuracies_all = []
val_losses_all = []

for i in range(num_runs):
    val_accuracies, val_losses = train_and_evaluate()
    val_accuracies_all.append(val_accuracies)
    val_losses_all.append(val_losses)
    print(f"Run {i + 1} completed")

val_accuracies_all = np.array(val_accuracies_all)
val_losses_all = np.array(val_losses_all)

average_val_accuracies = np.mean(val_accuracies_all, axis=0)
average_val_losses = np.mean(val_losses_all, axis=0)

# Plot results
plt.style.use("seaborn")
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 9))

ax1.plot(average_val_accuracies, "-b", label="With quantum convolution 2*2 kernel size")
ax1.set_ylabel("Accuracy")
ax1.set_ylim([0, 1])
ax1.set_xlabel("Epoch")
ax1.legend()

ax2.plot(average_val_losses, "-b", label="With quantum convolution 2*2 kernel size")
ax2.set_ylabel("Loss")
ax2.set_ylim(top=2.5)
ax2.set_xlabel("Epoch")
ax2.legend()
plt.tight_layout()
plt.show()


<IBMBackend('ibm_brisbane')>
<IBMBackend('ibm_kyoto')>
<IBMBackend('ibm_osaka')>
<IBMBackend('ibm_sherbrooke')>
Selected backend: <IBMBackend('ibm_kyoto')>
Quantum pre-processing of train images:
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has success

IBMBackendApiError: 'Error submitting job: \'403 Client Error: Forbidden for url: https://api.quantum.ibm.com/runtime/jobs. {"errors":[{"message":"Job create exceeds open plan job usage limits","code":4317,"solution":"Please wait until the beginning of next month to submit more jobs when your quota will reset.","more_info":"https://docs.quantum-computing.ibm.com/errors"}]}\''